In [9]:
import cv2, math, os, json, numpy as np
from pathlib import Path
from tqdm import tqdm
from pycocotools.coco import COCO

SRC_IMG_DIR   = Path('../data/rotation/batches/batch_20250703_01/images/default')
DEST_IMG_DIR  = Path('../data/rotation/batches/batch_20250703_01/images/boxes')   # patches land here
DEST_ANN_PATH = Path('../data/rotation/batches/batch_20250703_01/annotations/instances_train_rectified.json')



In [ ]:

PAD_RATIO = 1.05
ANGLES = [0, 45, 90, -45, -90, 180]

def rectify_patch(img, cx, cy, w, h, theta):
    """schneidet ein auf 0° korrigiertes Patch aus und gibt es zurück"""
    M = cv2.getRotationMatrix2D((cx, cy), -theta, 1.0)
    rot = cv2.warpAffine(
        img, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR
    )
    x0 = int(cx - w/2*PAD_RATIO); y0 = int(cy - h/2*PAD_RATIO)
    x1 = int(cx + w/2*PAD_RATIO); y1 = int(cy + h/2*PAD_RATIO)
    return rot[max(0,y0):y1, max(0,x0):x1]

def rotate_patch(patch, angle):
    """dreht Patch um angle (°) und gibt (rot_img, new_w, new_h) zurück"""
    h, w = patch.shape[:2]
    c  = (w/2, h/2)
    M  = cv2.getRotationMatrix2D(c, angle, 1.0)
    cos, sin = abs(M[0,0]), abs(M[0,1])
    new_w = int(h*sin + w*cos)
    new_h = int(h*cos + w*sin)
    M[0,2] += new_w/2 - c[0]
    M[1,2] += new_h/2 - c[1]
    rot = cv2.warpAffine(
        patch, M, (new_w, new_h),
        flags=cv2.INTER_LINEAR, borderValue=255     # Weißer Rand bei CAD
    )
    return rot, new_w, new_h
